In [1]:
import pandas as pd
import numpy as np
import unidecode

In [2]:
player_info = pd.read_csv('../dataset/player_info^(Stagione 2021_22).csv',index_col=[0])

In [3]:
player_info.shape

(548, 49)

In [4]:
player_xg = pd.read_csv('../dataset/player_xg_xa^(Stagione 2021_22).csv',index_col=[0]).drop_duplicates()

In [5]:
player_xg.shape

(570, 12)

In [6]:
player_xg.Team = player_xg.Team.apply(lambda x: x.split(',')[0] if len(x.split(','))==1 else x.split(',')[1])
player_xg.Team = player_xg.Team.apply(lambda x: x.replace('AC','').strip())

In [7]:
player_xg.Player = player_xg.Player.apply(lambda x:unidecode.unidecode(x))

In [8]:
player_xg = player_xg.drop_duplicates()

In [9]:
map_name = {'Ruslan Malinovskiy':'Ruslan Malinovskyi', 'Franck Zambo':'Zambo Anguissa André', 'Hamed Junior Traore':'Traorè Hamed ', 'Arthur':'Arthur Melo', 'Kelvin Adou':'Kelvin Amian',
           'Fabian':'Fabian Ruiz', 'Federico Di Francesco' : 'Di Francesco F', 'Jean-Daniel Akpa-Akpro':'Akpa Akpro Jean-Daniel', 'Luis Alberto':'Luis Alberto Alconchel', 'Marco Faraoni':'Davide Faraoni',
           'Petar Stojanovic':'Stojanovic Peter','Olivier Giroud':'Oliver Giroud','Mario Rui':'Mario Rui Duarte','Luiz Felipe':'Luiz Felipe Ramos','Joao Pedro':'Joao Pedro Galvao','Jose Reina':'Pepe Reina',
           'Juan Jesus':'Juan Jesus Nunes','Ibanez':'Roger Ibanez','Alex Sandro':'Alex Sandro Lobo','Danilo':'Danilo Luiz da Silva','Keita':'Keita Balde','Aleksey Miranchuk':'Miranchuk Aleksej',
           'Pedro':'Pedro Rodriguez Ledesma','Berat Gjimshiti':'Djimsiti Berat','Franck Ribery':'Frank Ribery','Juan Cuadrado':'Cuadrado Jaime','Jakub Kiwior':'Kiwior Jacub','Dimitris Nikolaou':'Nikolaou Dimitrios',
           'Di Francesco F':'Di Francesco Federico','Jose Callejon':'Josè Maria Callejon'}

In [10]:
def search_name(nome_xg,nome_info):
    
    array_info = nome_info.lower().replace('è','e').replace('ò','o').replace('à','a').replace('ì','i').replace('ù','u').split(' ')
    array_xg = nome_xg.lower().replace('è','e').replace('ò','o').replace('à','a').replace('ì','i').replace('ù','u').split(' ')
    
    if len(np.setdiff1d(array_info, array_xg)) == 0:
        return True
    else:
        return False

In [11]:
for player in player_xg.Player.unique():
    if player in map_name.keys():
        player_xg.loc[player_xg.Player == player,'Player'] = map_name[player]

In [12]:
player_info['index_merge'] = None
for nome_info in player_info.Nome.unique():
    merging = player_xg.Player.loc[player_xg.Player.apply(search_name, nome_info = nome_info)].index.tolist()
    if len(merging)>1:
        player_info.loc[player_info.Nome == nome_info, 'index_merge'] = str(merging)
    elif len(merging)>0:
        player_info.loc[player_info.Nome == nome_info, 'index_merge'] = merging[0]
    else:
        player_info.loc[player_info.Nome == nome_info, 'index_merge'] = None

In [13]:
final_data = player_info.merge(player_xg[['xG','xA','xG90','xA90','Performance_gol_segnati']], left_on='index_merge', right_index=True, how = 'left')

In [14]:
final_data[['xG','xA','xG90','xA90']] = final_data[['xG','xA','xG90','xA90']].fillna(0)
final_data['Performance_gol_segnati'] = final_data['Performance_gol_segnati'].fillna('No Data')

In [15]:
final_data.to_csv('../dataset/stat_info^(Stagione 2021_22).csv')

In [16]:
final_data.shape

(548, 55)